In [4]:
!pip install fastai -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.0 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have 

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import log_loss, roc_auc_score


import matplotlib.pyplot as plt
import seaborn as sns


from itertools import combinations
import random

from PIL import Image


from sklearn.metrics import roc_auc_score, log_loss

import gc

import torch
from fastai.tabular.all import *


import warnings
warnings.filterwarnings("ignore")




In [6]:
import pandas as pd
import numpy as np

# Set base path
base_path = "/kaggle/input/flood-prediction/data/"

# Load datasets
train = pd.read_csv(base_path + "Train.csv")
test = pd.read_csv(base_path + "Test.csv")
submission = pd.read_csv(base_path + "SampleSubmission.csv")
images = np.load(base_path + "composite_images.npz")

# Load train_with_cv_results with debugging
train_with_cv_results = pd.read_csv("/kaggle/input/flood-prediction/test_with_cv_results_accuracy_0_8931.csv")
print("Columns in train_with_cv_results:", train_with_cv_results.columns.tolist())
train_with_cv_results.columns = train_with_cv_results.columns.str.strip()  # Normalize column names
print("Normalized columns:", train_with_cv_results.columns.tolist())

# Select columns
try:
    train_with_cv_results = train_with_cv_results[['location_id', 'predicted_prob']]
    print("Successfully selected columns for train_with_cv_results")
except KeyError as e:
    print(f"KeyError: {e}")

# Load test_with_cv_results with debugging (fixed comma)
test_with_cv_results = pd.read_csv("/kaggle/input/flood-prediction/test_with_cv_results_accuracy_0_8931.csv")
print("Columns in test_with_cv_results:", test_with_cv_results.columns.tolist())
test_with_cv_results.columns = test_with_cv_results.columns.str.strip()
print("Normalized columns:", test_with_cv_results.columns.tolist())

# Select columns
try:
    test_with_cv_results = test_with_cv_results[['location_id', 'predicted_prob']]
    print("Successfully selected columns for test_with_cv_results")
except KeyError as e:
    print(f"KeyError: {e}")

# Display results
display(
    train.head(), train.shape,
    train_with_cv_results.head(), train_with_cv_results.shape,
    test.head(), test.shape
)

Columns in train_with_cv_results: ['location_id', 'event_id_counts', 'image_path', 'predicted_prob', 'predicted_class']
Normalized columns: ['location_id', 'event_id_counts', 'image_path', 'predicted_prob', 'predicted_class']
Successfully selected columns for train_with_cv_results
Columns in test_with_cv_results: ['location_id', 'event_id_counts', 'image_path', 'predicted_prob', 'predicted_class']
Normalized columns: ['location_id', 'event_id_counts', 'image_path', 'predicted_prob', 'predicted_class']
Successfully selected columns for test_with_cv_results


,event_id,precipitation,label
0,id_spictby0jfsb_X_0,0.000000,0
1,id_spictby0jfsb_X_1,0.095438,0
2,id_spictby0jfsb_X_2,1.949560,0
3,id_spictby0jfsb_X_3,3.232160,0
4,id_spictby0jfsb_X_4,0.000000,0


(492020, 3)

,location_id,predicted_prob
0,id_066zz28m11mr,0.000834
1,id_073l04ir88sn,0.998566
2,id_0ajr0br1b046,0.515890
3,id_0mdchsqsdw17,0.999215
4,id_0nu2nus290am,0.860427


(224, 2)

,event_id,precipitation
0,id_j7b6sokflo4k_X_0,0.00000
1,id_j7b6sokflo4k_X_1,3.01864
2,id_j7b6sokflo4k_X_2,0.00000
3,id_j7b6sokflo4k_X_3,16.61520
4,id_j7b6sokflo4k_X_4,2.56706


(163520, 2)

In [7]:
# List all keys in the .npz file
print("Keys in the .npz file:", images.files)

# Access the first image array (adjust the key name as needed)
key_name = images.files[0]  # Replace with the correct key if needed
image_array = images[key_name]

# Print the shape of the image
print("Shape of the image array:", image_array.shape)

# Extract the number of bands (assumes shape is (height, width, bands))
if image_array.ndim == 3:
    num_bands = image_array.shape[2]
    print("Number of bands in the image:", num_bands)
else:
    print("The image array does not have multiple bands (2D array).")

Keys in the .npz file: ['id_rhg5w8vmv3ny', 'id_rua8ey2jczl0', 'id_073l04ir88sn', 'id_wmkfqw7iwjmu', 'id_heri806er7xw', 'id_irft382nqvp0', 'id_ja1tvofxh403', 'id_2uyqsuz0anf7', 'id_h4hkhfp6ilg3', 'id_49o19mx5bbhc', 'id_g1tlrzptebsd', 'id_qg3xwgh1lfdd', 'id_595gz650j6u8', 'id_66akbt5q6bks', 'id_nmyx181qwsk3', 'id_stkvzvg77on0', 'id_1kctttnnhvjx', 'id_xk3049ov4j0u', 'id_f5e7zme7gqnb', 'id_61fh1d5zbsdr', 'id_sfedhmsqwbu4', 'id_oud1p540x3pe', 'id_iyay05bn5zwk', 'id_57cnhhuvhsxb', 'id_fi3md8cxzmp3', 'id_zgckftx7lo64', 'id_3f33h3wta3on', 'id_j7b6sokflo4k', 'id_m5wawccfyrn7', 'id_sww2v8tfqkrn', 'id_g0ydhtmd4ogx', 'id_aaf6d4a9hwq6', 'id_gt66wk2twhn3', 'id_8n80txb7wwsb', 'id_t437fhetcaf4', 'id_flkg8jcrzmrp', 'id_u5e7kr73e7k1', 'id_5i6fbm6h7y4e', 'id_pk95ni8fdeaq', 'id_vdzsn80c9e7r', 'id_f8pw6e0tebe5', 'id_j7moaw4bpjkp', 'id_omhl52u0te84', 'id_3ntxqfjssoln', 'id_5sq85iwd29vq', 'id_4e2iscrtbn5p', 'id_59uqistnu2di', 'id_0nu2nus290am', 'id_ukly29b2hds3', 'id_qgcksf3v8071', 'id_n07bvjlz6ekg', 'id_e2s

In [8]:
def get_location(value):
  return value.split("_")[0] + '_' + value.split("_")[1]

def get_event_id(value):
  return value.split("_")[3]
for df in [train, test]:

  df['location_id'] = df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
  df['event_idx'] = df.groupby('location_id', sort=False).ngroup()

  df['event_t'] = df.groupby('location_id').cumcount()

print(len(set(train['location_id'])), len(set(test['location_id'])))
print(len(set(train['location_id']).intersection(set(test['location_id']))))
print(len(images))
display(train.head(), test.head())

674 224
0
898


,event_id,precipitation,label,location_id,event_idx,event_t
0,id_spictby0jfsb_X_0,0.000000,0,id_spictby0jfsb,0,0
1,id_spictby0jfsb_X_1,0.095438,0,id_spictby0jfsb,0,1
2,id_spictby0jfsb_X_2,1.949560,0,id_spictby0jfsb,0,2
3,id_spictby0jfsb_X_3,3.232160,0,id_spictby0jfsb,0,3
4,id_spictby0jfsb_X_4,0.000000,0,id_spictby0jfsb,0,4


,event_id,precipitation,location_id,event_idx,event_t
0,id_j7b6sokflo4k_X_0,0.00000,id_j7b6sokflo4k,0,0
1,id_j7b6sokflo4k_X_1,3.01864,id_j7b6sokflo4k,0,1
2,id_j7b6sokflo4k_X_2,0.00000,id_j7b6sokflo4k,0,2
3,id_j7b6sokflo4k_X_3,16.61520,id_j7b6sokflo4k,0,3
4,id_j7b6sokflo4k_X_4,2.56706,id_j7b6sokflo4k,0,4


* each image has 730 events
* no intersection of images betweeen the two data sets (unique sets)
* The numpy files has 898 images for both train and test

* The images are annual cloud-free composite images from Sentinel-2 satellite imagery. They are of size 128x128 and contain the following 6 channels:

      Sentinel-2 B2 (Blue)
      Sentinel-2 B3 (Green)
      Sentinel-2 B4 (Red)
      Sentinel-2 B8 (NIR)
      Sentinel-2 B11 (SWIR)
      Slope (derived from NASA SRTM)

* the images are essentially static for any event/location pair over the study period.
  * the images only serve as spatial representations of the environment for that location over the 730 day period
  * it reflects static or semi-static environmental conditons (e.g land use, vegetation, water bodies, topography) that could influence flood occurence
  * so the images cannot provide temporal insights but what we can do is extract spatial features such as NDVI, NDWI, NDBI, Topographic features like slope and elevation changes from the slope channel
  * combine the spatial features with temporal precipitation data to enrich the dataset by treating the spatial features as fixed covariates that describe each location.
    * Areas with high NDWI Might flood more frequently with heavy precipitation
    * LOcations with high slope values might experience flash floods after intense rainfall

  * Image processing:
    * Use pretrained models to extract image embeddings or use PCA for dimensionality reduction
    * create a binary classifier where 1 is images where a flood has occured in any of the 730 events and 0 if no floods has occured to create a soft flag for flood-prone locations. Even if not perfect they can serve as a proxy for environmental vulnerability to floods
    * The image classifier naturally reduces the extreme imbalance in the dataset by focusing on binary flood/non-flood classification
  
  * clustering locations:
    * group events/locations based on spatial features (e.g NDVI, NDWI) to identify patterns in flood susceptibility
  * correlating spatial features with precipitation thresholds:
    * study how spatial features interact with specific precipitation thresholds that leads to floods

  * You can think of event_id_X_1 being the 01/01/2024 and event_id_X_2 being 02/01/2024 (dd/mm/yyyy).

### Data Preprocessing + Feature Engineering

In [9]:
import numpy as np
import pandas as pd
from scipy.ndimage import uniform_filter

def calculate_statistics(array, stats):
    """
    Compute specified statistics on a given array.

    Args:
        array (numpy.ndarray): Input array.
        stats (list): List of statistics to calculate (e.g., ['mean', 'std', 'median']).

    Returns:
        dict: Dictionary of computed statistics with keys as stat names.
    """
    statistics = {}
    for stat in stats:
        if stat == 'mean':
            statistics['mean'] = np.nanmean(array)
        elif stat == 'std':
            statistics['std'] = np.nanstd(array)
        elif stat == 'median':
            statistics['median'] = np.nanmedian(array)
        elif stat == 'skew':
            statistics['skew'] = pd.Series(array.flatten()).skew()
        elif stat == 'min':
            statistics['min'] = np.nanmin(array)
        elif stat == 'max':
            statistics['max'] = np.nanmax(array)
    return statistics


def generate_flood_features(df, images, band_names, statistics=['mean', 'std']):
    """
    Generate tabular features for flood modeling from given bands, with customizable statistics.

    Args:
        df (pd.DataFrame): Input dataframe containing metadata for each location (train + test).
                           Should have a 'location_id' column to identify images.
        images (dict): Dictionary of image data indexed by 'location_id'.
        band_names (tuple): Tuple of available band names (e.g., 'B2', 'B3', ...).
        statistics (list): List of statistics to calculate for each feature (e.g., ['mean', 'std']).

    Returns:
        pd.DataFrame: Dataframe with additional features for modeling.
    """
    band_indices = {name: band_names.index(name) for name in band_names}
    features = []

    for _, row in df.iterrows():
        location_id = row['location_id']
        bands = {name: images[location_id][..., idx] for name, idx in band_indices.items()}

        # Vegetation Indices
        ndvi = (bands['B8'] - bands['B4']) / (bands['B8'] + bands['B4'] + 1e-6)
        # savi = (1.5) * (bands['B8'] - bands['B4']) / (bands['B8'] + bands['B4'] + 0.5 + 1e-6)

        # Water Indices
        ndwi = (bands['B3'] - bands['B8']) / (bands['B3'] + bands['B8'] + 1e-6)
        mndwi = (bands['B3'] - bands['B11']) / (bands['B3'] + bands['B11'] + 1e-6)

        # Moisture-related
        msi = bands['B11'] / (bands['B8'] + 1e-6)

        # Topographic Features
        slope = bands['SLOPE']
        # slope_variability = uniform_filter(slope, size=3)  # Smooth with a 3x3 window
        # elevation_change = np.gradient(slope, axis=(0, 1))

        # # Composite Ratios
        # band_ratios = [
        #     (bands['B2'] / (bands['B4'] + 1e-6)),
        #     (bands['B8'] / (bands['B11'] + 1e-6)),
        #     (bands['B2'] / (bands['B8'] + 1e-6)),
        # ]

        # Compile features for this location
        location_features = {'location_id': location_id}

        # Add statistics for each feature
        index_features = {
            'NDVI': ndvi,
            # 'SAVI': savi,
            'NDWI': ndwi,
            'MNDWI': mndwi,
            'MSI': msi,
            'Slope': slope,
            # 'Slope_variability': slope_variability,
            # 'Elevation_change': elevation_change,
        }

        for feature_name, feature_array in index_features.items():
            stats = calculate_statistics(feature_array, statistics)
            for stat_name, value in stats.items():
                location_features[f'{feature_name}_{stat_name}'] = value

        # # Add statistics for band ratios
        # for idx, ratio in enumerate(band_ratios):
        #     stats = calculate_statistics(ratio, statistics)
        #     for stat_name, value in stats.items():
        #         location_features[f'Band_Ratio_{idx + 1}_{stat_name}'] = value

        features.append(location_features)

    # Convert to a DataFrame
    feature_df = pd.DataFrame(features)

    # Merge with the original DataFrame (optional)
    df = df.merge(feature_df, on='location_id', how='left')

    return df

# Example list of statistics to calculate
stats_to_calculate = ['mean', 'median', 'std']#, 'std', 'median', 'min', 'max'
BAND_NAMES = ('B2', 'B3', 'B4', 'B8', 'B11', 'SLOPE')

# Generate features with these statistics
train_features = generate_flood_features(train_with_cv_results, images, BAND_NAMES, statistics=stats_to_calculate)
test_features = generate_flood_features(test_with_cv_results, images, BAND_NAMES, statistics=stats_to_calculate)

display(train_features.head(), train_features.shape)

,location_id,predicted_prob,NDVI_mean,NDVI_median,NDVI_std,NDWI_mean,NDWI_median,NDWI_std,MNDWI_mean,MNDWI_median,MNDWI_std,MSI_mean,MSI_median,MSI_std,Slope_mean,Slope_median,Slope_std
0,id_066zz28m11mr,0.000834,0.081579,0.079895,0.023026,21.865115,21.777871,1.964856,17.325085,17.322720,2.070897,1.417197,1.416527,0.096802,2629.709961,2088.0,2034.574526
1,id_073l04ir88sn,0.998566,0.705933,0.053005,3.226964,17.513283,17.754353,2.889530,15.245295,15.145348,2.034187,1.275078,1.292132,0.154106,1972.433838,2089.0,1532.969214
2,id_0ajr0br1b046,0.515890,0.276265,0.277380,0.038207,25.633296,25.589490,1.850851,22.374170,22.542337,1.260905,1.217898,1.223550,0.087719,2268.098755,2087.0,1514.195685
3,id_0mdchsqsdw17,0.999215,1.637706,0.126988,5.110717,18.241311,19.372281,5.735027,17.522368,18.019794,4.379248,1.146777,1.141928,0.184854,1684.768005,2079.0,1409.701125
4,id_0nu2nus290am,0.860427,0.436188,0.444184,0.132800,22.802521,21.997103,4.231390,26.596867,25.513295,7.103207,0.840122,0.832346,0.168959,10377.869995,9164.0,5532.644748


(224, 17)

In [10]:
train_df = pd.merge(train, train_features, on='location_id', how='left')
test_df = pd.merge(test, test_features, on='location_id', how='left')


display(train_df.head(), train_df.shape, test_df.head(), test_df.shape)

,event_id,precipitation,label,location_id,event_idx,event_t,predicted_prob,NDVI_mean,NDVI_median,NDVI_std,...,NDWI_std,MNDWI_mean,MNDWI_median,MNDWI_std,MSI_mean,MSI_median,MSI_std,Slope_mean,Slope_median,Slope_std
0,id_spictby0jfsb_X_0,0.000000,0,id_spictby0jfsb,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,id_spictby0jfsb_X_1,0.095438,0,id_spictby0jfsb,0,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,id_spictby0jfsb_X_2,1.949560,0,id_spictby0jfsb,0,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,id_spictby0jfsb_X_3,3.232160,0,id_spictby0jfsb,0,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,id_spictby0jfsb_X_4,0.000000,0,id_spictby0jfsb,0,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(492020, 22)

,event_id,precipitation,location_id,event_idx,event_t,predicted_prob,NDVI_mean,NDVI_median,NDVI_std,NDWI_mean,...,NDWI_std,MNDWI_mean,MNDWI_median,MNDWI_std,MSI_mean,MSI_median,MSI_std,Slope_mean,Slope_median,Slope_std
0,id_j7b6sokflo4k_X_0,0.00000,id_j7b6sokflo4k,0,0,0.999893,0.31329,0.115435,1.661317,19.048946,...,3.508979,18.08611,17.734073,3.109924,1.132556,1.153117,0.191815,1735.735657,2079.0,1412.857797
1,id_j7b6sokflo4k_X_1,3.01864,id_j7b6sokflo4k,0,1,0.999893,0.31329,0.115435,1.661317,19.048946,...,3.508979,18.08611,17.734073,3.109924,1.132556,1.153117,0.191815,1735.735657,2079.0,1412.857797
2,id_j7b6sokflo4k_X_2,0.00000,id_j7b6sokflo4k,0,2,0.999893,0.31329,0.115435,1.661317,19.048946,...,3.508979,18.08611,17.734073,3.109924,1.132556,1.153117,0.191815,1735.735657,2079.0,1412.857797
3,id_j7b6sokflo4k_X_3,16.61520,id_j7b6sokflo4k,0,3,0.999893,0.31329,0.115435,1.661317,19.048946,...,3.508979,18.08611,17.734073,3.109924,1.132556,1.153117,0.191815,1735.735657,2079.0,1412.857797
4,id_j7b6sokflo4k_X_4,2.56706,id_j7b6sokflo4k,0,4,0.999893,0.31329,0.115435,1.661317,19.048946,...,3.508979,18.08611,17.734073,3.109924,1.132556,1.153117,0.191815,1735.735657,2079.0,1412.857797


(163520, 21)

### More Feature Engineering

In [11]:
from types import new_class
def apply_expanding_combinations(df, group_cols_list, target_col='Sales', shift_periods=[1], min_periods=1, stats=['mean', 'std']):
    # Loop through the group column combinations
    for group_cols in group_cols_list:
        # Generate base name for the grouping
        group_name = '_'.join(group_cols)

        for shift_period in shift_periods:
            for stat in stats:
                expanding_col_name = f'expanding_grouped_{group_name}_{target_col}_shift_{shift_period}_{stat}'

                # Apply groupby, shift, and expanding for the given statistic
                df[expanding_col_name] = df.groupby(group_cols)[target_col].transform(
                    lambda x: x.shift(shift_period).expanding(min_periods=min_periods).agg(stat)
                )

    return df

def smoothen_target(df, group_cols, target_col):
  n_std = 10
  for i_smooth in [target_col]:
      df_id_outlier = df.groupby(group_cols,as_index=False).agg({
          f'{i_smooth}': lambda x: x.mean() + n_std*x.std()
      }).rename(columns={f'{i_smooth}':f'{i_smooth}_outlier'})

      df_id_mean = df.groupby(group_cols,as_index=False).agg({
          f'{i_smooth}': 'mean'
      }).rename(columns={f'{i_smooth}':f'{i_smooth}_mean'})

      df = df.merge(df_id_outlier, on=group_cols[0], how='left')
      df = df.merge(df_id_mean, on=group_cols[0], how='left')

      df[f'{i_smooth}'] = np.where(
          df[f'{i_smooth}'] > df[f'{i_smooth}_outlier'],
          df[f'{i_smooth}_mean'],
          df[f'{i_smooth}']
      )

  return df


def create_rolling_features(data, group_cols, target_col, windows, shift_period, min_period, statistics):
    def apply_statistic(x, stat):
        rolled = x.shift(shift_period).rolling(window=window, min_periods=min_period)
        if stat == 'mean':
            return rolled.mean()
        elif stat == 'median':
            return rolled.median()
        elif stat == 'std':
            return rolled.std()
        elif stat == 'min':
            return rolled.min()
        elif stat == 'max':
            return rolled.max()
        elif stat == 'skew':
            return rolled.skew()
        elif stat == 'sum':
            return rolled.sum()
        elif stat == 'quantile':
            return rolled.quantile(0.95)

        elif stat.startswith('quantile_'):
            q = float(stat.split('_')[1])
            return rolled.quantile(q)
        else:
            raise ValueError(f"Unknown statistic: {stat}")

    for window in windows:
        for stat in statistics:
            stat_name = stat if not stat.startswith('quantile_') else f"{stat.split('_')[1]}th"
            col_name = f'rolling_previous_grouped_{target_col}_{stat_name}_{window}_{shift_period}'

            data[col_name] = data.groupby(group_cols)[target_col].transform(
                lambda x: apply_statistic(x, stat)
            )

    return data



def custom_agg(x):
    return x.max() - x.min()

def get_date_features(df):
  # Simulate year (assuming 365 days per year)
  df['year'] = (df['event_t'] // 365) + 1  # Year 1 or 2

  # Simulate month (approximate)
  df['month'] = ((df['event_t'] % 365) // 30) + 1  # 30-day months approximation

  # Simulate week of the year
  df['week_of_year'] = (df['event_t'] % 365) // 7 + 1

  # Simulate day of the month
  df['day_of_month'] = (df['event_t'] % 30) + 1  # Assuming 30-day months

  # Simulate day of the week (0 = Monday, 6 = Sunday)
  df['day_of_week'] = df['event_t'] % 7

  # Simulate quarter
  df['quarter'] = ((df['month'] - 1) // 3) + 1
  return  df




def feature_engineering(train, test):
  data = pd.concat([train, test])
  data.sort_values(by = ['location_id', 'event_t'], inplace=True)
  data['event_t'] = data['event_t'].astype(int)
  # data = smoothen_target(data, ['location_id'], 'precipitation')

  data['event_binary'] = data['event_t'].apply(lambda x: 1 if (x >= 296 and x <= 435) else 0)


  group_cols =['location_id']
  # data = apply_expanding_combinations(
  #     data,
  #     [group_cols],
  #     target_col='precipitation',
  #     shift_periods=[1],#1,3, 4, 5, 6, 7, 8, 24
  #     min_periods=1,
  #     stats=['mean']
  # )

  statistics = ['mean'] #, 'median', 'std', 'quantile_0.25', 'quantile_0.75'
  min_period = 1

  shift_period = 0
  windows = [3, 4,10,20, 25, 30,55,60, 75, 296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 2
  # windows = [3, 4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 3
  # windows = [3, 4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 4
  # windows = [3, 4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 5
  # windows = [3, 4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 6
  # windows = [ 3,4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)

  # shift_period = 8
  # windows = [ 3,4,10,20, 25, 30,55,60, 75,296]#3, 4, 10, 20,25,30, 50, 55, 60, 75,
  # data = create_rolling_features(data, group_cols,'precipitation', windows, shift_period, min_period, statistics)
  # # data = get_date_features(data)

  for col in ['precipitation']:
    # data[f"grouped_location_{col}_cum"] = data.groupby('location_id')[col].cumsum().shift(1)

    # quantile = 0.95  # Define the quantile you want to calculate
    # for stat in ['mean', 'quantile']:
    #     if stat != 'quantile':
    #         data[f"location_grouped_{col}_{stat}"] = data.groupby('location_id')[col].transform(stat)
    #         data[f"diff_{col}_{stat}"] = data[col] - data[f"location_grouped_{col}_{stat}"]


    for shift in range(1,365):
      data[f'{col}_shift_{shift}'] = data.groupby('location_id')[col].shift(shift)
      data[f'{col}_next_shift_{shift}'] = data.groupby('location_id')[col].shift(-shift)




    # for window in windows:
    #   data[f'{col}_rolling_grouped_custom_{window}'] = (
    #       data.groupby('location_id')[col]
    #       .rolling(window)
    #       .apply(custom_agg)
    #       .reset_index(level=0, drop=True)  # Reset the index to align with the original DataFrame
    #   )

    for span in [7]:
        data[f'{col}_ewm_grouped_mean_{span}'] = (
            data.groupby('location_id')[col]
            .ewm(span=span, adjust=False)
            .mean()
            .reset_index(level=0, drop=True)  # Reset the index to align it with the original DataFrame
        )




  train = data[data['label'].notna()].reset_index(drop = True)
  test = data[data['label'].isna()].reset_index(drop = True)

  return train, test

new_train, new_test = feature_engineering(train_df, test_df)
display(new_train.head(), new_train.shape, new_test.head(), new_test.shape)

,event_id,precipitation,label,location_id,event_idx,event_t,predicted_prob,NDVI_mean,NDVI_median,NDVI_std,...,precipitation_next_shift_360,precipitation_shift_361,precipitation_next_shift_361,precipitation_shift_362,precipitation_next_shift_362,precipitation_shift_363,precipitation_next_shift_363,precipitation_shift_364,precipitation_next_shift_364,precipitation_ewm_grouped_mean_7
0,id_05v6zjuaf300_X_0,0.00000,0.0,id_05v6zjuaf300,435,0,NaN,NaN,NaN,NaN,...,0.00000,NaN,0.00000,NaN,0.00000,NaN,0.00000,NaN,6.01065,0.000000
1,id_05v6zjuaf300_X_1,9.45176,0.0,id_05v6zjuaf300,435,1,NaN,NaN,NaN,NaN,...,0.00000,NaN,0.00000,NaN,0.00000,NaN,6.01065,NaN,24.95070,2.362940
2,id_05v6zjuaf300_X_2,0.00000,0.0,id_05v6zjuaf300,435,2,NaN,NaN,NaN,NaN,...,0.00000,NaN,0.00000,NaN,6.01065,NaN,24.95070,NaN,0.00000,1.772205
3,id_05v6zjuaf300_X_3,0.00000,0.0,id_05v6zjuaf300,435,3,NaN,NaN,NaN,NaN,...,0.00000,NaN,6.01065,NaN,24.95070,NaN,0.00000,NaN,0.00000,1.329154
4,id_05v6zjuaf300_X_4,0.00000,0.0,id_05v6zjuaf300,435,4,NaN,NaN,NaN,NaN,...,6.01065,NaN,24.95070,NaN,0.00000,NaN,0.00000,NaN,0.00000,0.996865


(492020, 762)

,event_id,precipitation,label,location_id,event_idx,event_t,predicted_prob,NDVI_mean,NDVI_median,NDVI_std,...,precipitation_next_shift_360,precipitation_shift_361,precipitation_next_shift_361,precipitation_shift_362,precipitation_next_shift_362,precipitation_shift_363,precipitation_next_shift_363,precipitation_shift_364,precipitation_next_shift_364,precipitation_ewm_grouped_mean_7
0,id_066zz28m11mr_X_0,0.0,NaN,id_066zz28m11mr,118,0,0.000834,0.081579,0.079895,0.023026,...,0.605245,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0
1,id_066zz28m11mr_X_1,0.0,NaN,id_066zz28m11mr,118,1,0.000834,0.081579,0.079895,0.023026,...,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0
2,id_066zz28m11mr_X_2,0.0,NaN,id_066zz28m11mr,118,2,0.000834,0.081579,0.079895,0.023026,...,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0
3,id_066zz28m11mr_X_3,0.0,NaN,id_066zz28m11mr,118,3,0.000834,0.081579,0.079895,0.023026,...,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0
4,id_066zz28m11mr_X_4,0.0,NaN,id_066zz28m11mr,118,4,0.000834,0.081579,0.079895,0.023026,...,0.000000,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0


(163520, 762)

### MODELLING
674 224

In [12]:
new_train['label'].value_counts()

label
0.0    491702
1.0       318
Name: count, dtype: int64

In [13]:

n_splits = 10
seed = 2024
gkf = StratifiedGroupKFold(n_splits = n_splits)

new_train['fold'] = -1
for fold, (_, val_idx) in enumerate(gkf.split(new_train, new_train['label'], groups = new_train['location_id'])):
    new_train.loc[val_idx, "fold"] = fold
# new_train['fold'] = new_train['fold'].astype(int)
new_train['fold'].value_counts()



fold
2    49640
0    49640
6    49640
1    49640
9    48910
4    48910
8    48910
7    48910
3    48910
5    48910
Name: count, dtype: int64

In [14]:
for i in range(n_splits):
  print(new_train[new_train['fold'] == i]['label'].value_counts())
  print("-"* 100)

label
0.0    49608
1.0       32
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
label
0.0    49608
1.0       32
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
label
0.0    49608
1.0       32
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
label
0.0    48879
1.0       31
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
label
0.0    48878
1.0       32
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
label
0.0    48879
1.0       31
Name: count, dtype: int64
----------------------------------------------------------------------------------------------------
label
0.0    49608
1.0       32
Name: count, d

### MODELLING

In [15]:

indices_cols = [
  'EVI_mean',
 'EVI_median',
 'EVI_std',
 'MNDWI_mean',
 'MNDWI_median',
 'MNDWI_std',
 'MSI_mean',
 'MSI_median',
 'MSI_std',
 'NDVI_mean',
 'NDVI_median',
 'NDVI_std',
 'NDWI_mean',
 'NDWI_median',
 'NDWI_std',
 'Slope_mean',
 'Slope_median',
 'Slope_std',
]

selected_columns =['precipitation','predicted_prob','event_binary', 'event_t', ] + [col for col in new_train if 'diff' in col or 'shift' in col or 'grouped' in col ]

print(selected_columns)
target_col = 'label'


['precipitation', 'predicted_prob', 'event_binary', 'event_t', 'rolling_previous_grouped_precipitation_mean_3_0', 'rolling_previous_grouped_precipitation_mean_4_0', 'rolling_previous_grouped_precipitation_mean_10_0', 'rolling_previous_grouped_precipitation_mean_20_0', 'rolling_previous_grouped_precipitation_mean_25_0', 'rolling_previous_grouped_precipitation_mean_30_0', 'rolling_previous_grouped_precipitation_mean_55_0', 'rolling_previous_grouped_precipitation_mean_60_0', 'rolling_previous_grouped_precipitation_mean_75_0', 'rolling_previous_grouped_precipitation_mean_296_0', 'precipitation_shift_1', 'precipitation_next_shift_1', 'precipitation_shift_2', 'precipitation_next_shift_2', 'precipitation_shift_3', 'precipitation_next_shift_3', 'precipitation_shift_4', 'precipitation_next_shift_4', 'precipitation_shift_5', 'precipitation_next_shift_5', 'precipitation_shift_6', 'precipitation_next_shift_6', 'precipitation_shift_7', 'precipitation_next_shift_7', 'precipitation_shift_8', 'precipi

In [18]:

def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)
 #cpu vars
    torch.manual_seed(seed_value)
# cpu  vars
    random.seed(seed_value)
 # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
# gpu vars
        torch.backends.cudnn.deterministic = True
 #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

random_seed(2024,True)
# Ensure previous imports are present: pandas, numpy, torch, gc, fastai.tabular.all, sklearn.metrics etc.
from fastai.tabular.all import *
from sklearn.metrics import roc_auc_score, log_loss
import gc
import torch
import numpy as np
import pandas as pd

# Assume random_seed is defined and called as before

def fit_fastai_model(train, test, target_col, selected_columns, n_splits):
    # Make sure 'oof_fastai' exists before trying to assign to it within loc
    if 'oof_fastai' not in train.columns:
        train['oof_fastai'] = 0.0
    if 'fastai_preds' not in test.columns:
        test['fastai_preds'] = 0.0

    test_preds_accum = np.zeros(len(test), dtype=np.float32) # Accumulator for test preds
    scores_auc = []
    scores_logloss = []

    cat_feats = [] # No categorical features specified
    cont_feats = [col for col in selected_columns if col not in cat_feats]

    for fold in range(n_splits):
        print("*" * 100)
        print(f"======================================TRAINING FOLD: {fold}=============================================")

        # Split train into training and validation sets for the current fold
        training_idx = train[train['fold'] != fold].index
        validation_idx = train[train['fold'] == fold].index

        # --- Explicit NaN Handling ---
        print(f"Fold {fold}: Checking and filling NaNs in selected columns...")
        # Create copies to avoid modifying the original DataFrame slices directly if not intended
        training_fold = train.loc[training_idx].copy()
        validation_fold = train.loc[validation_idx].copy()
        test_fold = test.copy() # Copy test data for consistent processing if needed later

        # Check initial NaNs (optional but good for debugging)
        # print("NaNs in training_fold[selected_columns]:\n", training_fold[selected_columns].isnull().sum().sort_values(ascending=False).head())
        # print("NaNs in validation_fold[selected_columns]:\n", validation_fold[selected_columns].isnull().sum().sort_values(ascending=False).head())

        # Fill NaNs - Using 0 is common for shift/lag features, but consider if another value (e.g., -1, median) is more appropriate
        # Apply filling to both training and validation folds based on training data where appropriate (e.g., median)
        # For simplicity here, we fill with 0, assuming it makes sense for the engineered features.
        training_fold[selected_columns] = training_fold[selected_columns].fillna(0)
        validation_fold[selected_columns] = validation_fold[selected_columns].fillna(0)
        # Also fill NaNs in the test set using the same strategy
        test_fold[selected_columns] = test_fold[selected_columns].fillna(0)

        # Check for infinite values (optional but recommended)
        inf_cols_train = training_fold[cont_feats].columns[np.isinf(training_fold[cont_feats]).any()]
        inf_cols_val = validation_fold[cont_feats].columns[np.isinf(validation_fold[cont_feats]).any()]
        if len(inf_cols_train) > 0 or len(inf_cols_val) > 0:
            print(f"Warning: Infinite values found in columns (Train: {inf_cols_train.tolist()}, Val: {inf_cols_val.tolist()}). Replacing with 0.")
            # Replace inf/-inf with a large number or 0, depending on context. 0 is simpler here.
            training_fold.replace([np.inf, -np.inf], 0, inplace=True)
            validation_fold.replace([np.inf, -np.inf], 0, inplace=True)
            test_fold.replace([np.inf, -np.inf], 0, inplace=True)


        # --- End of NaN Handling ---

        # Define splits based on the indices of the original combined DataFrame idea
        # Need to map original indices to the new combined DataFrame indices
        len_train_fold = len(training_fold)
        len_val_fold = len(validation_fold)
        splits = (list(range(len_train_fold)), list(range(len_train_fold, len_train_fold + len_val_fold)))

        # Combine the *filled* training and validation data for TabularPandas processing
        combined_data = pd.concat(
            [training_fold[selected_columns + [target_col]],
             validation_fold[selected_columns + [target_col]]],
            ignore_index=True # Important to reset index for splits to work correctly
        )

        # Prepare DataLoaders using the explicitly filled data
        # We still include FillMissing in procs as a safeguard, but most NaNs should be gone.
        # It might handle edge cases or fill based on training median more accurately than simple fill(0).
        procs = [Categorify, FillMissing, Normalize]
        dls = TabularPandas(
            combined_data,
            procs=procs,
            cat_names=cat_feats,
            cont_names=cont_feats,
            y_names=target_col,
            splits=splits,
            y_block=CategoryBlock() if train[target_col].nunique() == 2 else RegressionBlock() # Assuming binary target needs CategoryBlock
        ).dataloaders(bs=4096, num_workers=0) # Set num_workers=0 as recommended by seed function comment

        # Define the model
        # For binary classification with BCEWithLogitsLoss, n_out=1 is correct.
        learn = tabular_learner(
            dls,
            layers=[256, 512, 1024, 512, 256], # Example layers
            n_out=1, # Single output logit for binary classification
            loss_func=BCEWithLogitsLossFlat(), # Use BCEWithLogitsLossFlat for binary target
            metrics=[accuracy, RocAucBinary()] # Use RocAucBinary for binary tasks
            # Consider adding weight_decay or other regularization in config if needed
        )

        # Train the model
        # Find LR (optional but recommended)
        # suggested_lr = learn.lr_find(suggest_funcs=(valley, slide))[0] # Get the suggested LR
        # print(f"Fold {fold} Suggested LR: {suggested_lr}")
        # Use a fixed LR or the found one
        learn.fit_one_cycle(
            10, # Number of epochs
            2e-3, # Learning rate (or use suggested_lr)
            cbs=[
                 SaveModelCallback(monitor='valid_loss', fname=f'fastai_tabular_fold_{fold}'),
                 EarlyStoppingCallback(monitor='valid_loss', patience=3) # Add early stopping
                 ]
            )

        # Validation predictions
        # Use the validation_fold DataFrame which has the correct original indices
        val_dl = learn.dls.test_dl(validation_fold[selected_columns])
        preds, _ = learn.get_preds(dl=val_dl)
        # Output is logits, apply sigmoid. Squeeze if shape is (N, 1)
        val_preds = torch.sigmoid(preds).squeeze().numpy()

        # Ensure val_preds is 1D array
        if val_preds.ndim > 1:
             val_preds = val_preds.squeeze()
        if len(validation_fold[target_col]) != len(val_preds):
            print(f"Warning: Length mismatch in Fold {fold} validation. Labels: {len(validation_fold[target_col])}, Preds: {len(val_preds)}")
        else:
            try:
                auc_score = roc_auc_score(validation_fold[target_col], val_preds)
                # Clip preds for log_loss
                val_preds_clipped = np.clip(val_preds, 1e-15, 1 - 1e-15)
                logloss_score = log_loss(validation_fold[target_col], val_preds_clipped)
                scores_auc.append(auc_score)
                scores_logloss.append(logloss_score)
                print(f"Fold {fold} AUC: {auc_score:.4f}, LogLoss: {logloss_score:.4f}")
                # Assign OOF predictions using original validation indices
                train.loc[validation_idx, 'oof_fastai'] = val_preds
            except ValueError as e_score:
                 print(f"Error calculating scores for Fold {fold}: {e_score}")
                 print(f"Labels unique values: {validation_fold[target_col].unique()}")
                 print(f"Preds min/max: {np.min(val_preds)}, {np.max(val_preds)}")


        # Test predictions for the current fold
        # Use the filled test_fold DataFrame
        test_dl = learn.dls.test_dl(test_fold[selected_columns])
        preds, _ = learn.get_preds(dl=test_dl)
        test_preds_fold = torch.sigmoid(preds).squeeze().numpy()
        if test_preds_fold.ndim > 1:
             test_preds_fold = test_preds_fold.squeeze()
        test_preds_accum += test_preds_fold # Accumulate predictions

        # Cleanup
        del dls, learn, val_dl, test_dl, preds, _, training_fold, validation_fold, test_fold, combined_data
        gc.collect()
        torch.cuda.empty_cache()

    # Finalize test predictions (average across folds)
    combined_test_preds = test_preds_accum / n_splits

    print("-" * 100)
    print(f"\nAverage AUC across {n_splits} folds: {np.mean(scores_auc):.4f} (+/- {np.std(scores_auc):.4f})")
    print(f"Average LogLoss across {n_splits} folds: {np.mean(scores_logloss):.4f} (+/- {np.std(scores_logloss):.4f})")

    # Overall scores for OOF - Ensure NaNs weren't assigned to oof_fastai
    oof_preds_final = train['oof_fastai'].fillna(0.5) # Fill potential NaNs if scoring failed on a fold
    try:
        overall_auc = roc_auc_score(train[target_col], oof_preds_final)
        overall_logloss = log_loss(train[target_col], np.clip(oof_preds_final, 1e-15, 1 - 1e-15))
        print(f"Overall OOF AUC: {overall_auc:.4f}, LogLoss: {overall_logloss:.4f}")
    except ValueError as e_final_score:
        print(f"Error calculating final OOF scores: {e_final_score}")


    # Assign combined test predictions to the original test DataFrame
    test['fastai_preds'] = combined_test_preds

    return train, test

In [ ]:
sub_train, sub_test =  fit_fastai_model(new_train, new_test, target_col, selected_columns, n_splits)

****************************************************************************************************
======================================TRAINING FOLD: 0=============================================
Fold 0: Checking and filling NaNs in selected columns...



Average AUC across 10 folds: 0.9386 (+/- 0.0274)
Average LogLoss across 10 folds: 0.0027 (+/- 0.0003)
Overall OOF AUC: 0.9310, LogLoss: 0.0027

In [ ]:
display(sub_test.head(), sub_test.shape)

In [ ]:

sub = sub_test[['event_id', 'fastai_preds']]
sub.head()

In [ ]:
sub.to_csv("baseline_fastai_0.0027_10_folds_v2.csv", index = False)

#### Normalizing the Probabilities

In [ ]:
from sklearn.metrics import log_loss

print(f"logloss before normalizing: {log_loss(sub_train['label'], sub_train['oof_fastai'])}")

locations_to_normalize = sub_train[sub_train['predicted_prob'] >= 0.5]['location_id'].unique()
sub_train['oof_sum_prob'] = sub_train.groupby('location_id')['oof_fastai'].transform('sum')

# Avoid division by zero
epsilon = 1e-8
sub_train['oof_fastai_norm'] = sub_train['oof_fastai']  # Copy original values

sub_train.loc[sub_train['location_id'].isin(locations_to_normalize), 'oof_fastai_norm'] = (
    sub_train.loc[sub_train['location_id'].isin(locations_to_normalize), 'oof_fastai'] /
    (sub_train.loc[sub_train['location_id'].isin(locations_to_normalize), 'oof_sum_prob'] + epsilon)
)

print(f"logloss after normalizing: {log_loss(sub_train['label'], sub_train['oof_fastai_norm'])}")


In [ ]:
sub_train[['event_id', 'location_id', 'event_t', 'predicted_prob','label','oof_fastai', ]].to_csv("fastai_train_with_oof.csv", index=False)
sub_test[['event_id', 'location_id', 'event_t', 'predicted_prob','label', 'fastai_preds']].to_csv("fastai_test_with_oof.csv", index=False)

### Normalize the predictions based on the flood probability


In [ ]:
locations_to_normalize = sub_test[sub_test['predicted_prob'] >= 0.7]['location_id'].unique()
sub_test['oof_sum_prob'] = sub_test.groupby('location_id')['fastai_preds'].transform('sum')

# Avoid division by zero
epsilon = 1e-8
sub_test['pred_norm'] = sub_test['fastai_preds']  # Copy original values

sub_test.loc[sub_test['location_id'].isin(locations_to_normalize), 'pred_norm'] = (
    sub_test.loc[sub_test['location_id'].isin(locations_to_normalize), 'fastai_preds'] /
    (sub_test.loc[sub_test['location_id'].isin(locations_to_normalize), 'oof_sum_prob'] + epsilon)
)

sub_test.head()

In [ ]:

mod_sub = sub_test[['event_id', 'pred_norm']]
mod_sub.head()

In [ ]:
mod_sub.to_csv("fastai_mod_0_25(corrected).csv", index = False)